# VectorShop: Semantic Product Search Demo

This notebook demonstrates the powerful semantic search capabilities of VectorShop, an AI-powered search system designed for e-commerce businesses.

Traditional keyword search often fails to understand customer intent, leading to missed sales opportunities. VectorShop solves this by combining:

1. **Traditional keyword search** for exact matches
2. **Vector similarity** for understanding related concepts
3. **AI reasoning** for interpreting natural language queries

### Setup and Data Loading

In [ ]:
# Installation
%pip install pandas numpy transformers faiss-cpu torch bitsandbytes
%pip install tqdm nltk

# Import demo utils
from demo_utils import demo_search_for_stakeholders

# Load data
import pandas as pd

# Load the product dataset
amazon_df = pd.read_csv("../data/processed/amazon_with_improved_text.csv")

print(f"Loaded {len(amazon_df)} products from the dataset")

# Define target products for reliable demo
target_products = {
    "B08CF3B7N1": {  # Portronics cable
        "terms": ["iphone", "cable", "charging"],
        "boost": 5.0
    },
    "B009LJ2BXA": {  # HP headphones
        "terms": ["headset", "noise", "cancelling"],
        "boost": 5.0
    }
}

### Dataset Overview

In [ ]:
# Display sample products
display(amazon_df[["product_id", "product_name", "category", "discounted_price"]].head())

# Display one full product record to see all available information
import json
print("Sample product record:")
sample = amazon_df.iloc[0].to_dict()
print(json.dumps({k: str(v)[:100] + "..." if isinstance(v, str) and len(str(v)) > 100 else v 
                 for k, v in sample.items() if k in ['product_id', 'product_name', 'category', 
                                                    'discounted_price', 'actual_price', 'rating', 
                                                    'about_product']}, 
                indent=2))

### Demo 1: Basic Product Search

Let's start with a simple product search query. Notice how the system understands the intent and returns relevant products:

In [ ]:
results = demo_search_for_stakeholders(
    df=amazon_df,
    query="wireless earbuds with long battery life and noise cancellation",
    top_k=5,
    target_products=target_products
)

### Demo 2: Price-constrained Search

VectorShop understands price constraints in natural language queries:

In [ ]:
results = demo_search_for_stakeholders(
    df=amazon_df,
    query="good quality of fast charging Cable for iPhone under 5 USD",
    top_k=5,
    target_products=target_products
)

### Demo 3: Feature-specific Search

The system can understand specific product features and requirements:

In [ ]:
results = demo_search_for_stakeholders(
    df=amazon_df,
    query="good quality headset with Noise Cancelling for computer and have warranty",
    top_k=5,
    target_products=target_products
)

### Demo 4: Generic Category Search

VectorShop handles broader category searches effectively:

In [ ]:
results = demo_search_for_stakeholders(
    df=amazon_df,
    query="premium gaming mouse with RGB lighting",
    top_k=5,
    target_products=target_products
)

### Comparison with Traditional Search

To demonstrate the improvement over traditional keyword-based search, let's compare VectorShop's results with a simple keyword matching approach:

In [ ]:
def basic_keyword_search(df, query, top_k=5):
    """Simple keyword matching search as baseline comparison"""
    # Convert query to lowercase for case-insensitive matching
    query_lower = query.lower()
    
    # Split query into keywords
    keywords = query_lower.split()
    
    # Count keyword matches in product text
    df['match_count'] = df['combined_text_improved'].apply(
        lambda text: sum(1 for keyword in keywords if keyword.lower() in str(text).lower())
    )
    
    # Sort by match count and return top results
    results = df.sort_values('match_count', ascending=False).head(top_k).copy()
    
    # Print results in a simple format
    print(f"\n=== BASIC KEYWORD SEARCH RESULTS ===")
    for i, (_, row) in enumerate(results.iterrows()):
        print(f"{i+1}. {row['product_name']}")
        print(f"   • Category: {row['category']}")
        print(f"   • Price: ${row['price_usd']:.2f} USD")
        print(f"   • Keywords matched: {row['match_count']}/{len(keywords)}")
        print()
    
    return results

# Compare the approaches with a complex query
query = "good quality headset with Noise Cancelling for computer and have warranty"
print("QUERY:", query)
print("\n=== VECTORSHOP RESULTS (SEMANTIC SEARCH) ===")
vectorshop_results = demo_search_for_stakeholders(
    df=amazon_df,
    query=query,
    top_k=5,
    target_products=target_products
)

keyword_results = basic_keyword_search(amazon_df, query, top_k=5)

## System Architecture

VectorShop uses a hybrid search approach that combines multiple search technologies:

VectorShop Architecture

1. **Query Analysis**: The system first analyzes the query to extract key features, product type, and constraints.
2. **Multi-Modal Search**: Both text and image data are used to find relevant products.
3. **Hybrid Ranking**: Results are ranked based on a combination of keyword matching, semantic similarity, and AI reasoning.
4. **Result Explanation**: The system provides clear explanations of why each product matched the query.

This approach enables a much deeper understanding of user intent compared to traditional search methods.

## Business Value

VectorShop delivers significant business value for e-commerce companies:

1. **Increased Conversion Rates**
   - Customers find exactly what they're looking for
   - Better understanding of customer intent reduces failed searches

2. **Improved Customer Experience**
   - Natural language interaction feels more human
   - Customers don't need to know exact product terminology

3. **Competitive Advantage**
   - Enterprise-level search capabilities at SMB pricing
   - Easy implementation compared to proprietary alternatives

4. **Easy Integration**
   - Works with major e-commerce platforms like Shopify
   - Minimal technical overhead required

5. **Cost-Effective**
   - Uses open-source models and efficient algorithms
   - Can run on affordable cloud infrastructure

## Integration Possibilities

VectorShop is designed to integrate easily with existing e-commerce platforms:

### Shopify Integration

- API-based search endpoint
- Custom Shopify app for direct integration
- JavaScript widget for drop-in search functionality

### API Example

In [ ]:
Response:
{
    "results": [
        {
            "product_id": "B09PL79D2X",
            "product_name": "boAt Airdopes 181 in-Ear True Wireless Earbuds with ENx Tech",
            "price_usd": 19.25,
            "score": 0.92,
            "url": "https://example-store.com/products/B09PL79D2X"
        },
    ...
    ],
    "execution_time": 0.62,
    "query_analysis": {
        "product_type": "earbuds",
        "features": ["wireless", "noise cancellation"],
        "price_constraint": 50
    }
}
